In [ ]:
import os
import csv
import logging as lg
import mysql.connector as connection

# removing the log file if it already exists with the same name
if os.path.exists('database.log'):
    os.remove('database.log')

lg.basicConfig(filename = 'database.log', level = lg.INFO,format = '%(asctime)s - %(message)s')

console_log = lg.StreamHandler()
console_log.setLevel(lg.DEBUG)
format = lg.Formatter('%(asctime)s - %(message)s')
console_log.setFormatter(format)
lg.getLogger('').addHandler(console_log)
lg.info('Logging Started')

try:    
    conn = connection.connect(host = 'localhost', 
                              user = 'root', 
                              database ='nilesh_db' , 
                              passwd = 'mysql', 
                              use_pure = True)
    if conn.is_connected():
        lg.info('Establishing connection.....')
        lg.info('Connection Established with the database!!')
        
        cur = conn.cursor()
        cur.execute('DROP TABLE IF EXISTS carbon_nanotubes')
        
        lg.info('Creating Table.......')
        cur.execute(
            "create table carbon_nanotubes (Chiral_indice_n INT (7),"
                                            "Chiral_indice_m INT (7)," 
                                            "Initial_atomic_coordinate_u INT (7),"
                                            "Initial_atomic_coordinate_v INT (7),"
                                            "Initial_atomic_coordinate_w INT (7),"
                                            "Calculated_atomic_coordinates_u INT (7),"
                                            "Calculated_atomic_coordinates_v INT (7),"
                                            "Calculated_atomic_coordinates_w INT (7))"
                )
        lg.info('Table created!!')
        
        with open('carbon_nanotubes.csv',"r") as file:
            lg.info('Starting record Insertion.......\n')
            next(file)
            carbon_data = csv.reader(file,delimiter = '\n')

            for num,record in enumerate(carbon_data,start = 1):
                rec = record[0].split(';')
                
                # converting the elements of the list into integer and removing the initial zeros present in the csv
                record_processed = [int(i.split(',')[1]) if len(i.split(',')) > 1 else int(i) for i in rec]
                
                # converting the list into comma-separated string which will serve as input to INSERT statement
                record_processed = ','.join([str(elem) for elem in record_processed]) 
                
                cur.execute(
                            "insert into carbon_nanotubes values({values})".format(values = record_processed)
                           )
                lg.info(f'Record {num} inserted')
                
        lg.info('All records inserted!!')
        conn.commit()
        
except Exception as e:
    lg.error(str(e))
    conn.close()
    
finally:
    if conn.is_connected():
        cur.close()
        conn.close()
        lg.info("Connection closed")
        lg.shutdown()